In [ ]:
'''
Detectron2:
    - It is from facebook AI Reasearch team (FAIR)
    - Pytorch based Project
    - Includes new capabilities such as panoptic segmentation, Densepose, Cascade R-CNN, rotated bounding boxes, 
      PointRend, DeepLa
    - Using Single library we can do
        1. Object Detection
        2. Image Segmentation
        3. Panoptic Segmentation
        4. Key point Detection
        
Setup Detectron2 on windows 
https://www.youtube.com/watch?v=Pb3opEFP94U

In [ ]:
'''
Setup Detectron2 on windows 10

    C:\Users\bmi_cims>conda create -n detectron_env python=3.8

    C:\Users\bmi_cims>conda activate detectron_env

    (detectron_env) C:\Users\bmi_cims>conda install pytorch==1.8.0 torchvision==0.9.0 torchaudio==0.8.0 cpuonly -c pytorch

    **** conda install pytorch torchvision torchaudio cudatoolkit=11.0 -c pytorch  (If u are using cuda)

    (detectron_env) C:\Users\bmi_cims>pip install cython

    (detectron_env) C:\Users\bmi_cims>git clone https://github.com/facebookresearch/detectron2.git

    (detectron_env) C:\Users\bmi_cims>cd C:\Users\bmi_cims\detectron2

    (detectron_env) C:\Users\bmi_cims\detectron2>pip install -e .

    (detectron_env) C:\Users\bmi_cims\detectron2>pip install opencv-python

    (detectron_env) C:\Users\bmi_cims\detectron2>jupyter notebook

In [ ]:
'''
Detectron2 Object detection Example on Google Colab

!pip install pyyaml==5.1

!pip install torch==1.7.1+cu101 torchvision==0.8.2+cu101 torchaudio==0.7.2 -f https://download.pytorch.org/whl/torch_stable.html

!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.7/index.html


# import some common detectron2 utilities
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

from detectron2 import model_zoo
import cv2
from google.colab.patches import cv2_imshow

image = cv2.imread("/content/ppl.jpg")
cv2_imshow(image)

cfg = get_cfg()

# Object Detection

cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml"))
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml")

# Segmentation

cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_X_101_32x8d_FPN_3x.yaml"))
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_X_101_32x8d_FPN_3x.yaml")

# Key Point Detection

cfg.merge_from_file(model_zoo.get_config_file("COCO-Keypoints/keypoint_rcnn_X_101_32x8d_FPN_3x.yaml"))
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Keypoints/keypoint_rcnn_X_101_32x8d_FPN_3x.yaml")

# panoptic Segementation

cfg.merge_from_file(model_zoo.get_config_file("COCO-PanopticSegmentation/panoptic_fpn_R_101_3x.yaml"))
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-PanopticSegmentation/panoptic_fpn_R_101_3x.yaml")

cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.6

predictor = DefaultPredictor(cfg)
outputs = predictor(image)

v = Visualizer(image[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
output = v.draw_instance_predictions(outputs["instances"].to("cpu"))
cv2_imshow(output.get_image()[:, :, ::-1])

In [ ]:
'''

Detectron2 on Custom Dataset

*** Note: Download the dataset and Save it on Google drive
Here we are having dataset on the path '/content/drive/MyDrive/Dataset/data'

******************************

!pip install pyyaml==5.1

******************************

!pip install torch==1.7.1+cu101 torchvision==0.8.2+cu101 torchaudio==0.7.2 -f https://download.pytorch.org/whl/torch_stable.html

******************************

!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.7/index.html

******************************

from google.colab import drive
drive.mount('/content/gdrive')

******************************

import detectron2

from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import matplotlib.pyplot as plt
import numpy as np
import cv2
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2 import model_zoo

******************************

%cd /content/drive/MyDrive/Dataset/data/images

******************************

from detectron2.data.datasets import register_coco_instances
register_coco_instances("fruits_nuts", {}, "/content/drive/MyDrive/Dataset/data/trainval.json", 
                                           "/content/drive/MyDrive/Dataset/data/images")

******************************

fruits_nuts_metadata = MetadataCatalog.get("fruits_nuts")
dataset_dicts = DatasetCatalog.get("fruits_nuts")

******************************

import random
for d in random.sample(dataset_dicts, 3):
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=fruits_nuts_metadata, scale=0.5)
    vis = visualizer.draw_dataset_dict(d)
    cv2_imshow(vis.get_image()[:, :, ::-1])

******************************

%cd /content/drive/MyDrive/Dataset/data/images

******************************
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg

import os

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))

cfg.DATASETS.TRAIN = ("fruits_nuts",)
cfg.DATASETS.TEST = ()   # no metrics implemented for this dataset
cfg.DATALOADER.NUM_WORKERS = 2

# initialize from model zoo
cfg.MODEL.WEIGHTS = "detectron2://COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/137849600/model_final_f10217.pkl"  


cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.02
cfg.SOLVER.MAX_ITER = 300    # 300 iterations seems good enough, but you can certainly train longer
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128   # faster, and good enough for this toy dataset
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 3  # 3 classes (data, fig, hazelnut)

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

******************************

cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5   # set the testing threshold for this model
cfg.DATASETS.TEST = ("fruits_nuts", )
predictor = DefaultPredictor(cfg)

******************************

from detectron2.utils.visualizer import ColorMode

for d in random.sample(dataset_dicts, 3):    
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)
    v = Visualizer(im[:, :, ::-1],
                   metadata=fruits_nuts_metadata, 
                   scale=0.8, 
                   instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels
    )
    v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    cv2_imshow(v.get_image()[:, :, ::-1])
    
******************************

In [ ]:
'''
Detectron2 Configuration  Hyperparameters

from detectron2.config import get_cfg

#Specify Model yaml & weights to grab
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_C4_1x.yaml"))
cfg.MODEL.WEIGHTS = "/home/svidelock/source/Detectron/model_final_721ade.pkl" # Let training initialize from model zoo 

#Spcify DIR for output, if not specified will create "output" DIR
# cfg.OUTPUT_DIR = '/home/svidelock/source/Detectron/HyperParamDetectron/output2/'

#Specify Datasets
cfg.DATASETS.TRAIN = ("boat_train",) #list of the pre-computed proposal files for trianing
cfg.DATASETS.TEST = ("boat_val",) #validation set

#Hyperparams
cfg.SOLVER.IMS_PER_BATCH = 2 #means that in 1 iteration the model sees 2 images 
cfg.SOLVER.BASE_LR = 0.02 #learning rate

#Some other configurable items
cfg.DATALOADER.NUM_WORKERS = 2 # depends on harware config ... 
cfg.SOLVER.WARMUP_ITERS = 1000 #constant learning rate
cfg.SOLVER.STEPS = (1000, 1500) #Decaying learning rate
cfg.SOLVER.GAMMA = 0.001 # The iteration number to decrease learning rate by GAMMA
cfg.SOLVER.MAX_ITER = 500 # Model will stop after this many iterations
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128 #look into
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # only has one class (boat)

#specify if CPU Training
cfg.MODEL.DEVICE='cpu'#cpu training

#Checkpoint/ValidationSet Params
cfg.TEST.EVAL_PERIOD = 20 # Tests validation set every 20 itterations
cfg.SOLVER.CHECKPOINT_PERIOD = cfg.TEST.EVAL_PERIOD #saves a checkpoint model each time we validate 

In [ ]:
'''
Inference of Detectron2 model in local system

from detectron2.utils.logger import setup_logger
setup_logger()
import cv2
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

config_file_path = "E:\detectron2_webapp_trained_on_custom_dataset\config.yml"

weights_path = "E:\detectron2_webapp_trained_on_custom_dataset\model_final.pth"

image_path = "E:\detectron2_webapp_trained_on_custom_dataset\test_image"

model = config_file_path
im = cv2.imread(image_path)
cfg = get_cfg()
cfg.merge_from_file(config_file_path)

cfg.MODEL.WEIGHTS = weights_path
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.8

predictor = DefaultPredictor(cfg)
outputs = predictor(im)

MetadataCatalog.get(cfg.DATASETS.TRAIN[0]).thing_classes = ['Apple','Oranges']

v = Visualizer(im[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
cv2.imshow('',v.get_image()[:, :, ::-1])
cv2.waitKey(0)